In [64]:
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt

from error_meas import error_meas

In [65]:
meas = error_meas()

images_dir = "data/20210508_full_cw_show"
csv_file = images_dir + "/data.csv"
df_raw = pd.read_csv(csv_file)

In [66]:
df_mod = df_raw.copy()
for index, row in df_mod.iterrows():
    image_path = images_dir + "/" + df_mod.at[index, "image_name"]
    image = cv2.imread(image_path)
    if image is None:
        df_mod.drop([index], inplace=True)
df_mod.reset_index(drop=True, inplace=True)

In [67]:
desired_angle = []
error_pos = []
for index, row in df_mod.iterrows():
    image = cv2.imread(images_dir + "/" + df_mod.at[index, "image_name"])
    angle, pos = meas.implement(image)
    desired_angle.append(angle)
    error_pos.append(pos)
df_mod["desired_angle"] = desired_angle
df_mod["error_pos"] = error_pos

f:\Neural_Network_Steering_Control\source_code\lane_helper\error_meas.py:230: RankWarning: Polyfit may be poorly conditioned
  self.left_fit, self.right_fit, self.left_lane_inds, self.right_lane_inds, self.visualization_data = self.sliding_window_1st_deg(self.cropped_image, self.histogram, self.leftx_base, self.rightx_base)


In [68]:
raw_filename = df_mod.at[0, "image_name"]
print(raw_filename)
filename = raw_filename.split("_")[1][:-7]
print(filename)
filename = raw_filename.split("_")[1][4:6]
print(filename)

20210508_163544599574.png
163544599
44


In [69]:
def strTime2ms(str_time):
    # print(str_time)
    str_time = str_time[:-3]
    # print(str_time)
    cum_time = int(str_time[6:])
    # print(cum_time)
    cum_time = cum_time + 1000*int(str_time[4:6])
    cum_time = cum_time + 60*1000*int(str_time[2:4])
    cum_time = cum_time + 24*60*1000*int(str_time[0:2])
    return cum_time
# time = strTime2ms(raw_filename.split("_")[1][:-4])
# print(time)

In [70]:
### check ###
index = 0
now_filename = df_mod.at[index, "image_name"]
next_filename = df_mod.at[index+1, "image_name"]
now_time = strTime2ms(now_filename.split("_")[1][:-4])
print(now_filename.split("_")[1][:-4])
print(now_time)
next_time = strTime2ms(next_filename.split("_")[1][:-4])
print(next_filename.split("_")[1][:-4])
print(next_time)
# time = (next_time - int(df_mod.at[index+1, "process_time"])) - (now_time - int(df_mod.at[index, "process_time"]))
time = next_time - now_time
print(time)

163544599574
25184599
163544721016
25184721
122


In [71]:
period = []
for index, row in df_mod.iterrows():
    if index == len(df_mod.index)-1:
        period.append(0)
    else:
        now_filename = df_mod.at[index, "image_name"]
        next_filename = df_mod.at[index+1, "image_name"]
        now_time = strTime2ms(now_filename.split("_")[1][:-4])
        next_time = strTime2ms(next_filename.split("_")[1][:-4])
        # time = (next_time - int(df_mod.at[index+1, "process_time"])) - (now_time - int(df_mod.at[index, "process_time"]))
        time = next_time - now_time
        period.append(time)
df_mod["period"] = period

In [72]:
df_mod.to_csv(images_dir + ".csv", index = False)